In [15]:
# Author: Ben Grauer 
# Created a python script that does some basic exploratory analysis / dov and exports to excel for easy viewing
# Originally designed for exploratory analysis of the open lending club data set from Kaggle

####################
# Log
####################
# 02/00/2017 - Script Created 
# 04/00/2017 - Added more summary stats, included additional tabs
# 05/00/2017 - Re-did stats to include variance, pivoted the data to allow sorting in excel. Added function pullSummaryStats_df 
#            - Froze some column headers.  Sample head(75) + tail(75)
# 06/00/2017 - created ordered summary stats for quick sorting - also in progress step of re-factoring the code
# 07/10/2017 - Fixed the orderd summary stats

In [16]:
# NEXT STEPS
# - for the correlation.  Put anything above .50 in an orange / light blue status.  Then anything more in deep red/blue
# - Put a notes section for whether to keep or not. 
# - Include a possibility for a list of columns to go in to keep.
# - If you have classifiers / Drivers, then show distribution against that (if a binary classification) - maybe save for plotting script?
# - add something that will indicate if there are major groupings (by percentage) - mainly want to see if groups are differnt
# - reg expression items
# - something that detects years (1900 - 2000), the word year or word month (with 75% success)
# - Min/Max on strings / dates
# - ?Insert Plots Images? - worksheet.insert_image('B2', 'python.png')

# INTELLIGENT NOTES
# -- mark > 25%, 40%, 70%, 90% nulls

# BUGS
# -- none so far!

# COMPLETED ITEMS
# X - Add conditional formatting to correlation to see highly correlated or negative correlated
# X - Need to add a variance to the stats summary data frame.  
# X - Identify the continous, vs descrete
# X - for n/a it is calculating percentage incorrectly - see mths_since_last_delinq

In [17]:
import xlsxwriter
import pandas as pd
from xlsxwriter.utility import xl_rowcol_to_cell
import numpy as np

In [23]:
# This function will pivot the summary stats (with fields in first colmn, and the stats other columns - to sort with)
# As time permits - need to clear out the first row - AND then comment out the dfNewStat, since we will not be using it.
#   we could have used it, but time-lag and just assigned each variable and then flipped it.  clean up the code
def pullSummaryStats_df(df):
    
    # dfOrdStat - is the fullly ordered stat list - by column
    # dfNewStat - is the newly dynamic created set
    
    # Create a new data frame for the numeric data types
    dfNumericTypes = df.select_dtypes(include=[np.number])

    # Set the columns we will use for the data frames
    columns = ['count','NaN','NaNPerc','mean','median','std','var','range','0%','25%','50%','75%','100%']
    
    dfOrdStat = pd.DataFrame(index=(['']), columns = columns )

    # for each column, grab stats
    for column in dfNumericTypes:
                
        # count
        statCount = np.round(dfNumericTypes[column].count(), 2)
        #NaN
        NaNCount = len(dfNumericTypes[column]) - dfNumericTypes[column].count()
        #NaN%
        NaNCountPerc = round(float((len(df) - df[column].count()) / len(df[column])), 4)
        # mean
        statMean = np.round(dfNumericTypes[column].mean(), 4)
        # median
        statMedian = dfNumericTypes[column].median()
        # Std
        statStd = dfNumericTypes[column].std()
        # Var
        statVar = dfNumericTypes[column].var()
        # Range
        statRange = dfNumericTypes[column].max() - dfNumericTypes[column].min()
        # min
        statMin = np.round(float(dfNumericTypes[column].quantile([0.0])), 4) 
        # 25%
        stat25 = np.round(float(dfNumericTypes[column].quantile([0.25])), 4)
        # 50%
        stat50 = np.round(float(dfNumericTypes[column].quantile([0.50])), 4)
        # 75%
        stat75 = np.round(float(dfNumericTypes[column].quantile([0.75])), 4)    
        # max
        statMax = np.round(float(dfNumericTypes[column].quantile([1.0])), 4)
        
        # Set the data (for the row), index and columns
        data = [(statCount, NaNCount, NaNCountPerc, statMean, statMedian, statStd, statVar, statRange, statMin, stat25, stat50, stat75, statMax)]
        indexName = ([dfNumericTypes[column].name])
        
        # 07/10/2017 - Ben Grauer - adjusting here
        # Create a new data frame of the data
        dfData = pd.DataFrame(data, index=indexName, columns = columns )
        
        # then concatenate / add on to the data set
        dfOrdStat = pd.concat([dfOrdStat, dfData])

    # return dfNewStat
    return dfOrdStat

In [24]:
# This is just a quick test of the function above

# Russia House - Main csv
df = pd.read_csv('/project/data/kg_RussiaHousing/test.csv')

# For testing the function above
dfTest = pullSummaryStats_df(df)
dfTest

,count,NaN,NaNPerc,mean,median,std,var,range,0%,25%,50%,75%,100%
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,7662.0,0.0,0.0000,3.430450e+04,3.430450e+04,2.211973e+03,4.892826e+06,7.661000e+03,3.047400e+04,3.238925e+04,3.430450e+04,3.621975e+04,3.813500e+04
full_sq,7662.0,0.0,0.0000,5.370120e+01,5.041000e+01,2.010432e+01,4.041837e+02,4.030000e+02,0.000000e+00,3.920000e+01,5.041000e+01,6.330000e+01,4.030000e+02
life_sq,6486.0,1176.0,0.1535,3.266000e+01,3.040000e+01,2.273786e+01,5.170101e+02,8.690000e+02,0.000000e+00,1.990000e+01,3.040000e+01,4.180000e+01,8.690000e+02
floor,7662.0,0.0,0.0000,7.652600e+00,7.000000e+00,5.099418e+00,2.600407e+01,4.000000e+01,1.000000e+00,4.000000e+00,7.000000e+00,1.100000e+01,4.100000e+01
max_floor,7662.0,0.0,0.0000,1.259110e+01,1.400000e+01,6.659291e+00,4.434616e+01,4.800000e+01,0.000000e+00,9.000000e+00,1.400000e+01,1.700000e+01,4.800000e+01
material,7662.0,0.0,0.0000,1.854200e+00,1.000000e+00,1.517171e+00,2.301809e+00,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,6.000000e+00
build_year,6613.0,1049.0,0.1369,1.820888e+03,1.982000e+03,5.529940e+02,3.058023e+05,2.019000e+03,0.000000e+00,1.966000e+03,1.982000e+03,2.011000e+03,2.019000e+03
num_room,7662.0,0.0,0.0000,1.876400e+00,2.000000e+00,8.356651e-01,6.983362e-01,1.800000e+01,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,1.900000e+01
kitch_sq,7662.0,0.0,0.0000,6.938700e+00,7.000000e+00,2.557969e+01,6.543205e+02,1.968000e+03,0.000000e+00,1.000000e+00,7.000000e+00,9.500000e+00,1.968000e+03


In [20]:
# Function to take a workbook, sheetname, and iterate through to populate columns
# TODO: Break out by number:  array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)
#      df.dtypes
def excel_add_df(inputWorkBook, inputWorkSheet, inputSheetName, inputDataFrame, 
                 inputStartRow=0,inputStartCol=0, inputUseIndex=False):
    
    bold = inputWorkBook.add_format({'bold': True})
    font14bold = inputWorkBook.add_format({'font_size':14, 'bold': True})
    
    # local variables
    rowNum = inputStartRow
    colNum = inputStartCol
    
    # add the input header about the sheet
    inputWorkSheet.write(rowNum, inputStartCol, inputSheetName, font14bold)
    rowNum = rowNum + 1
    
    # add an index
    if inputUseIndex==True:
        inputWorkSheet.write_column(inputStartRow+3, colNum, inputDataFrame.index, bold)
        colNum = colNum + 1
    
    for column in inputDataFrame:
        rowNum = inputStartRow + 2  # for the column headers. so 2 down
        inputWorkSheet.write(rowNum, colNum, inputDataFrame[column].name, bold)
        
        rowNum = inputStartRow + 3
        inputWorkSheet.write_column(rowNum, colNum, inputDataFrame[column])
    
        # add a column
        colNum = colNum + 1

In [21]:
import pylab
from pylab import figure, axes, pie, title, show

# open the excel sheet
def excel_gen_file(df, outputFileName):

    #workbook = xlsxwriter.Workbook('LendingClub_DOV.xlsx', {'nan_inf_to_errors': True})
    workbook = xlsxwriter.Workbook(outputFileName, {'nan_inf_to_errors': True})

    # Add the DOV Work sheet - quite a bit of code
    worksheet = workbook.add_worksheet('DOV')

    # Add a bold format to use to highlight cells.
    bold = workbook.add_format({'bold': True})
    italic = workbook.add_format({'italic': True})
    underline = workbook.add_format({'underline': True})
    formatHigh = workbook.add_format({'font_color': 'red'})
    formatLow = workbook.add_format({'font_color': 'blue'})

    offSetDataType = 1
    offSetNumNaN = 2
    offsetCount = 3
    offsetMean = 4
    offsetMedian = 5
    offsetStd = 6
    offsetVar = 7
    offsetRange = 8
    offset0Prct = 9
    offset25Prct = 10
    offset50Prct = 11
    offset75Prct =12
    offset100Prct = 13
    
    offsetContOrDesc = 15
    offsetNotes = 16
    
    offsetRowDataHeader = 18
    
    offsetRowFreezeRow = 18

    # star the column Iteration at 2
    colIteration = 1
    rowIteration = 0 # to start
    rowDataHeader = offsetRowDataHeader # bumpt out to 5 to start, so we have 4 for summary data

    # set the row descriptions
    worksheet.write(rowIteration, 0, 'Col Name', bold)
    worksheet.write(offSetDataType, 0, 'Data Type', bold)
    worksheet.write(offSetNumNaN, 0, '# Na', bold)
    worksheet.write(offsetCount, 0, 'count', bold)
    worksheet.write(offsetMean, 0, 'mean', bold)
    worksheet.write(offsetMedian, 0, 'median', bold)
    worksheet.write(offsetStd, 0, 'std', bold)
    worksheet.write(offsetVar, 0, 'var', bold)
    worksheet.write(offsetRange, 0, 'range', bold)
    worksheet.write(offset0Prct, 0, '0%', bold)
    worksheet.write(offset25Prct, 0, '25%', bold)
    worksheet.write(offset50Prct, 0, '50%', bold)
    worksheet.write(offset75Prct, 0, '75%', bold)
    worksheet.write(offset100Prct, 0, '100%', bold)
    
    worksheet.write(offsetContOrDesc, 0, 'Cont_Disc')
    worksheet.write(offsetNotes, 0, 'Notes')
    

    for column in df:
        worksheet.write(rowIteration, colIteration, df[column].name, bold) # header
        worksheet.write(offSetDataType, colIteration, str(df[column].dtypes)) # data type

        # TODO: # of na's 
        count_nan = len(df[column]) - df[column].count()
        
        if count_nan > 0:
            count_nan = str(round(( (len(df) - df[column].count()) / len(df[column]))*100, 0)) + '% - ' + str(count_nan)  


        worksheet.write(offSetNumNaN, colIteration, count_nan) # data type
        worksheet.write(offsetContOrDesc, colIteration, 'continous') # auto set continous - discrete determined below

        # This was the original stats area.  Possibly take from the existing data set, or move around / re-factor
        # if the columns is a numerical data type - give the summary stat
        if np.issubdtype(df[column].dtype, np.number):
        
            # count
            worksheet.write(offsetCount, colIteration, df[column].count())
            # mean
            worksheet.write(offsetMean, colIteration, df[column].mean())
            # median
            worksheet.write(offsetMedian, colIteration, df[column].median())
            # Std
            worksheet.write(offsetStd, colIteration, df[column].std())
            # Var
            worksheet.write(offsetVar, colIteration, df[column].var())
            # Range    
            worksheet.write(offsetRange, colIteration, df[column].max() - df[column].min())
            # min
            worksheet.write(offset0Prct, colIteration, df[column].quantile([0.0]))
            # 25%
            worksheet.write(offset25Prct, colIteration, df[column].quantile([0.25]))
            # 50%
            worksheet.write(offset50Prct, colIteration, df[column].quantile([0.50]))
            # 75%
            worksheet.write(offset75Prct, colIteration, df[column].quantile([0.75]))
            # max
            worksheet.write(offset100Prct, colIteration, df[column].quantile([1.0]))
        
        # header for the DOV
        worksheet.write(rowDataHeader-1, colIteration, 'DOV', underline)

        if df[column].nunique() > 500:
            worksheet.write(rowDataHeader, colIteration, '> 500 unq', italic)
            worksheet.write_column(rowDataHeader+1, colIteration, df[column].head(500))
            colIteration = colIteration + 1
            
        else:

            # here just saying if less than 25, then categorical vs continous - to move the dial
            if df[column].nunique() < 100:
                if np.issubdtype(df[column].dtype, np.number):
                    worksheet.write(rowIteration+offsetContOrDesc, colIteration, 'discrete') # data type
                else:
                    worksheet.write(rowIteration+offsetContOrDesc, colIteration, 'categorical') # data type
            
            # grab the distribution percentage
            disbDF = pd.DataFrame(df.groupby([column]).size() * 100 / len(df))
            disbDF.rename(columns={0:'distprc'}, inplace=True)
            disbDF = disbDF.sort_values(['distprc'], ascending=False)

            # write the distribution header + percentage

            # TODO: change the index name to "DOV" for better readability
            worksheet.write_column(rowDataHeader, colIteration, disbDF.index) # here we are adding the index which is the DOV
            colIteration = colIteration + 1

            worksheet.write(rowDataHeader-1, colIteration, 'DistPrc')
            worksheet.write_column(rowDataHeader, colIteration, disbDF.loc[: ,'distprc'])
            colIteration = colIteration + 1
    
    
        # Notes - here give a section for the note.
            # if all nulls, remove
            # if two parts of the distribution are above 10% - ok, if two are about 15% even better
            # if more than 90%, 95%, or 97% of all data is in a single category.

    worksheet.freeze_panes(offsetRowDataHeader, 1) # # Freeze the first row and column
        
            
    # Summary Stats - changed from df.describe() to new
    #worksheet = workbook.add_worksheet('SummaryStats')
    #excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='SummaryStats', 
    #             inputDataFrame=dfNewStat,inputStartRow=0,inputStartCol=0,inputUseIndex=True)
    
    dfNewStat = pullSummaryStats_df(df)
    worksheet = workbook.add_worksheet('OrderSummaryStats')
    excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='OrderSummaryStats', 
                 inputDataFrame=dfNewStat,inputStartRow=0,inputStartCol=0,inputUseIndex=True)
    

    # Write out the correlation matrix work-sheet
    worksheet = workbook.add_worksheet('Correlation')
    dfCorr = df.corr().round(2)
    excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='Correlation', 
                 inputDataFrame=dfCorr,inputStartRow=0,inputStartCol=0,inputUseIndex=True)

    # add conditional highlighting
    # length of columns
    lenColsDfCorr = len(dfCorr.columns) + 1
    lenRowsDfCorr = len(dfCorr.index) + 4 # for header
    cellStart = xl_rowcol_to_cell(3, 2)  # C2
    cellEnd = xl_rowcol_to_cell(len(dfCorr.index) + 2, len(dfCorr.columns) )  # C2
    worksheet.conditional_format(cellStart + ':' + cellEnd, {'type':     'cell',
                                            'criteria': '>=',
                                            'value':    0.75,
                                            'format':   formatHigh})
    worksheet.conditional_format('B3:K12', {'type':     'cell',
                                            'criteria': '<',
                                            'value':    -0.75,
                                            'format':   formatLow})

    
    # Write out the co-variance matrix work-sheet
    worksheet = workbook.add_worksheet('Co-Variance')
    excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='Co-Variance', 
                 inputDataFrame=df.cov().round(2),inputStartRow=0,inputStartCol=0,inputUseIndex=True)


    # Write out the 200 Samples - 100 head / 100 tail
    worksheet = workbook.add_worksheet('150samples')
    excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='150samples (Top 75)', 
                 inputDataFrame=df.head(75),inputStartRow=0,inputStartCol=0,inputUseIndex=True)
    
    excel_add_df(inputWorkBook=workbook, inputWorkSheet=worksheet, inputSheetName='150samples (Tail 75)', 
                 inputDataFrame=df.tail(75),inputStartRow=80,inputStartCol=0,inputUseIndex=True)

    
    # Test with inserting a plot - TODO: needs more work on types of plots to incorporate
    # Make a square figure and axes
    
    # figure(1, figsize=(7, 7))
    # ax = axes([0.1, 0.1, 0.8, 0.8])
    # labels = 'Frogs', 'Hogs', 'Dogs', 'Logs'
    # fracs = [15, 30, 45, 10]
    # explode = (0, 0.05, 0, 0)
    # pie(fracs, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True)
    # title('Raining Hogs and Dogs', bbox={'facecolor': '0.8', 'pad': 5})
    # pylab.savefig('/project/data/kg_RussiaHousing/foo.png', bbox_inches='tight')
    # worksheet = workbook.add_worksheet('Plots')
    # worksheet.insert_image('C3', '/project/data/kg_RussiaHousing/foo.png')
    
    
    #writer.save()
    workbook.close()

In [22]:
# Code / Portion of script that will take an input *.csv file and then output the file.
# The file open portion is designed to work on a Mac, other OS systems may differ
# Proces for testing each file read/excel generation
import subprocess, os

# Russia Housing - Macro
# inputFileName = '/project/data/kg_RussiaHousing/Macro.csv'
# outputFileName = '/project/data/kg_RussiaHousing/DOV_Macro.xlsx'

# Russia House - Main csv
# inputFileNme = '/project/data/kg_RussiaHousing/train.csv'
# outputFileName = '/project/data/kg_RussiaHousing/DOV_RsnHouse_Train.xlsx'

# Russia House - Main csv
inputFileNme = '/project/data/kg_RussiaHousing/test.csv'
outputFileName = '/project/data/kg_RussiaHousing/DOV_RsnHouse_Test.xlsx'

# Russia House - Full Data Set
# inputFileNme = '/project/data/kg_RussiaHousing/FullSet.csv'
# outputFileName = '/project/data/kg_RussiaHousing/DOV_FullDataSet.xlsx'

#outputFileName = '/project/data/kg_lendingclub/LendingClub_DOV.xlsx'
#outputFileName = '/Project/data/kg_titantic/DOV_Titantic.xlsx'

# Mercedes Benz Engineering
# inputFileNme = '/project/data/kg_Mercedes/train.csv'
# outputFileName = '/project/data/kg_Mercedes/DOV_train.xlsx'

########################
# Generate the excel file
# Read the input file
df = pd.read_csv(inputFileNme)
# gen the exel 
excel_gen_file(df, outputFileName)

print('Done!')

os.system("open "+outputFileName)


Done!


0